In [1]:
from preprocessing import *
from model import *

In [2]:
def get_data(path):
    X,y=load_data(path)

    # making a single column for all the activities
    y['activity']=y.idxmax(axis=1)

    # convering the categorical data into numerical data
    y['activity']=y['activity'].replace({'label:FIX_walking':0,'label:SITTING':1,'label:SLEEPING':2,'label:OR_standing':3,'label:FIX_running':4})

    y = y['activity']

    X['y'] = y

    return X

In [9]:
data = get_data("00EABED2-271D-49D8-B599-1D4A09240601.features_labels.csv.gz")

In [12]:
# list all data files with .gz extension
import glob
filenames = glob.glob("*.gz")
filenames

['11B5EC4D-4133-4289-B475-4E737182A406.features_labels.csv.gz',
 '27E04243-B138-4F40-A164-F40B60165CF3.features_labels.csv.gz',
 '5EF64122-B513-46AE-BCF1-E62AAC285D2C.features_labels.csv.gz',
 '7D9BB102-A612-4E2A-8E22-3159752F55D8.features_labels.csv.gz',
 '9DC38D04-E82E-4F29-AB52-B476535226F2.features_labels.csv.gz',
 '7CE37510-56D0-4120-A1CF-0E23351428D2.features_labels.csv.gz',
 '1DBB0F6F-1F81-4A50-9DF4-CD62ACFA4842.features_labels.csv.gz',
 '24E40C4C-A349-4F9F-93AB-01D00FB994AF.features_labels.csv.gz',
 '0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv.gz',
 '2C32C23E-E30C-498A-8DD2-0EFB9150A02E.features_labels.csv.gz',
 '00EABED2-271D-49D8-B599-1D4A09240601.features_labels.csv.gz',
 '4E98F91F-4654-42EF-B908-A3389443F2E7.features_labels.csv.gz',
 '0E6184E1-90C0-48EE-B25A-F1ECB7B9714E.features_labels.csv.gz',
 '4FC32141-E888-4BFF-8804-12559A491D8C.features_labels.csv.gz',
 '0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv.gz']

In [108]:
data = get_data(filenames[0])
print(data['y'].min(),data['y'].max())
data.to_csv('train.csv', index=False)
# print("data.csv created")
for i in filenames[1:-1]:
    data = get_data(i)
    print(data['y'].min(),data['y'].max())
    # appending all the data files into a single csv
    data.to_csv('train.csv', mode='a', header=False, index=False)
    # print(i+" appended")
data = get_data(filenames[-1])
print(data['y'].min(),data['y'].max())
data.to_csv('test.csv', index=False)

0 3
0 3
0 3
0 3
0 4
0 4
0 4
0 3
0 4
0 3
0 3
0 3
0 3
0 4
0 4


## now testing the files

In [109]:
data = pd.read_csv('train.csv')

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import json

# Define the PyTorch neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 5)

    def forward(self, x):
        x = nn.functional.leaky_relu(self.fc1(x))
        x = nn.functional.leaky_relu(self.fc2(x))
        x = nn.functional.leaky_relu(self.fc3(x))
        x = nn.functional.softmax(self.fc4(x), dim=1)
        return x
    
# Features and labels
X = data.drop('y', axis=1).values
y = data['y'].values

# standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# PyTorch data loaders
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.RMSprop(model.parameters(), lr=0.001, momentum=0.1)

for epoch in range(10):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

# convert all the model parameters to json
params = {}
for name, param in model.named_parameters():
    params[name] = param.data.numpy().tolist()

# save the model parameters to a json file
with open('trained13.json', 'w') as f:
    json.dump(params, f)

In [110]:
# Evaluate the model

# loading test data
test_data = pd.read_csv('test.csv')

predict(test_data, 'trained13.json')

('Test accuracy:', 0.1802018260451706)

In [112]:
re_train(test_data, 'trained13.json', 'trained14.json')

('Test accuracy:', 0.16546762589928057)